In [1]:
import os

import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from triton.language import tensor

from TSForecasting.data_provider.data_factory import data_provider
from train_model import train, test, predict
from TSForecasting.models.ConvTimeNet import Model


def preprocess_date(data):
    data = data.copy()

    data['date'] = pd.to_datetime(data['date'])
    data['day'] = data['date'].dt.day
    data['month'] = data['date'].dt.month
    data['year'] = data['date'].dt.year

    data['day_of_week'] = data['date'].dt.dayofweek
    data['week_of_year'] = data['date'].dt.isocalendar().week
    data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype(int)
    data['is_holiday'] = data['holiday'].astype(int)
    data['quarter'] = data['date'].dt.quarter

    return data


def preprocess_data(train, test):
    columns_to_have = list(test.columns) + ['orders']
    train = train[columns_to_have]

    # Preprocess date columns
    train = preprocess_date(train)
    test = preprocess_date(test)

    # Remove unwanted columns
    train = train.drop(['holiday_name', 'id'], axis=1)
    test = test.drop(['holiday_name', 'id'], axis=1)

    # Label Encoding
    le = LabelEncoder()
    train['warehouse'] = le.fit_transform(train['warehouse'])
    test['warehouse'] = le.transform(test['warehouse'])

    # making target (orders) column as the last column for df
    if 'orders' in train.columns:
        cols = [col for col in train.columns if col != 'orders']
        cols.append('orders')
        train = train[cols]
    else:
        print("Target column 'target' not found in DataFrame.")

    # Making date column as the first column of df
    if 'date' in train.columns:
        cols = [col for col in train.columns if col != 'date']
        cols = ['date'] + cols
        train = train[cols]
    else:
        print("Column 'date' not found in DataFrame.")

    if 'date' in test.columns:
        cols = [col for col in test.columns if col != 'date']
        cols = ['date'] + cols
        test = test[cols]
    else:
        print("Column 'date' not found in DataFrame.")

    return train, test


def save_model(model, model_name):
    print(f"Saving the model with model_name: {model_name}")

    if isinstance(model, torch.nn.DataParallel):
        model = model.module
    torch.save(model.state_dict(), model_name)

    print(f"Saving successfull!!!")


def load_model(model, model_name, device='cuda'):
    model.load_state_dict(torch.load(model_name))

    print(f"Model is set to eval() mode...")
    model.eval()

    print(f"Model is on the deivce: {device}")
    model.to(device)

    return model


def save_train_test_datasets(root_dir, data_path, preprocessing=False, save_data=False):
    train = pd.read_csv(os.path.join(root_dir, data_path, 'train.csv'))
    test = pd.read_csv(os.path.join(root_dir, data_path, 'test.csv'))

    if preprocessing:
        train, test = preprocess_data(train, test)
        print(f"Train Shape: {train.shape}")
        print(f"Test Shape: {test.shape}")

        print(f"Train set columns: {train.columns}")
        print(f"Test set columns: {test.columns}")

    # Creating a column called "orders" for test set. This won't be used.
    test['orders'] = 5000

    if save_data:
        print(f"Saving processed train and test files...")
        train.to_csv('data/processed/train_set_processed.csv', index=False)
        test.to_csv('data/processed/test_set_processed.csv', index=False)


args = {

    # Starters
    'root_dir': '.', 'data_path': 'data/',

    # For train, val and test sets, training data is split into 3 parts
    'train_data_path': 'data/processed/train_set_processed.csv', 'train_flag': 'train',
    'val_data_path': 'data/processed/train_set_processed.csv', 'val_flag': 'val',
    'test_data_path': 'data/processed/train_set_processed.csv', 'test_flag': 'test',

    # To score unseen data, we use new_data_flag = 'pred'
    'unseen_data_path': 'data/processed/test_set_processed.csv', 'unseen_data_flag': 'pred',
    'data': 'custom', 'features': 'MS', 'target': 'orders',
    'batch_size': 16, 'freq': 'd', 'seq_len': 14, 'label_len': 14, 'pred_len': 1,
    'embed': 'timeF',

    # Training params
    'checkpoints': './checkpoints/', 'patience': 5, 'use_amp': True, 'train_epochs': 2, 'learning_rate': 0.00001,

    # Model args
    # enc_in: Number of input features to the model
    # patch_ks: Kernel size for the patch-based convolutional layers.
    # patch_sd: Stride of the convolutional kernel for patches.
    # revin: Indicates whether to use reversible networks.
    # affine:  Whether to use affine transformations in normalization layers.
    # dw_ks: List of kernel sizes for depth-wise convolutions.
    # re_param:  Indicates whether to use a specific type of parameterization.
    # re_param_kernel:  Kernel size for the reparameterization.
    # enable_res_param: Whether to enable residual parameterization.
    # head_type: Type of output head used in the model.
    'enc_in': 14, 'e_layers': 6, 'd_model': 128, 'd_ff': 256, 'dropout': 0.05, 'head_dropout': 0.0,
    'patch_ks': 16, 'patch_sd': 3, 'padding_patch': 'end', 'revin': 1, 'affine': 0,
    'subtract_last': 0, 'dw_ks': [11, 15, 21, 29, 39, 51], 're_param': 1, 're_param_kernel': 3,
    'enable_res_param': 1, 'norm': 'batch', 'act': "gelu", 'head_type': 'flatten',

    # Test
    'test_flop': False, 'do_predict': True,

}


In [2]:
save_train_test_datasets(root_dir=args['root_dir'], data_path=args['data_path'],
                         preprocessing=True, save_data=True)

train_dataset, train_loader = data_provider(root_path=args['root_dir'], data_path=args['train_data_path'],
                                            flag=args['train_flag'], features=args['features'],
                                            target=args['target'], data=args['data'],
                                            batch_size=args['batch_size'], freq=args['freq'],
                                            seq_len=args['seq_len'], label_len=args['label_len'],
                                            pred_len=args['pred_len'], embed=args['embed'])

model = Model(enc_in=args['enc_in'], seq_len=args['seq_len'], pred_len=args['pred_len'],
              e_layers=args['e_layers'], d_model=args['d_model'], d_ff=args['d_ff'],
              dropout=args['dropout'], head_dropout=args['head_dropout'],
              patch_ks=args['patch_ks'], patch_sd=args['patch_sd'],
              padding_patch=args['padding_patch'], revin=args['revin'],
              affine=args['affine'], subtract_last=args['subtract_last'],
              dw_ks=args['dw_ks'], re_param=args['re_param'],
              re_param_kernel=args['re_param_kernel'],
              enable_res_param=args['enable_res_param'])

# model = train(args, model, train_dataset, train_loader, val_dataset, val_loader,
#               test_dataset, test_loader)
model = train(args, model, train_dataset, train_loader)

# Saving extra timesteps for evaluation
original_train_set = pd.read_csv('data/processed/train_set_processed.csv')
original_test_set = pd.read_csv('data/processed/test_set_processed.csv')

last_sequence = original_train_set.iloc[-args['seq_len']:, :]

modified_test_set = pd.concat([last_sequence, original_test_set], ignore_index=True)
modified_test_set.to_csv('data/processed/test_set_processed_modified.csv', index=False)

# unseen_data, unseen_loader = data_provider(root_path=args['root_dir'],
#                                            data_path='data/processed/test_set_processed_modified.csv',
#                                            flag='train', features=args['features'],
#                                            target=args['target'], data=args['data'],
#                                            batch_size=1, freq=args['freq'],
#                                            seq_len=args['seq_len'], label_len=args['label_len'],
#                                            pred_len=args['pred_len'], embed=args['embed']
#                                            )

# predict(args, model, unseen_data, unseen_loader)


Train Shape: (7340, 15)
Test Shape: (397, 14)
Train set columns: Index(['date', 'warehouse', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'day', 'month', 'year',
       'day_of_week', 'week_of_year', 'is_weekend', 'is_holiday', 'quarter',
       'orders'],
      dtype='object')
Test set columns: Index(['date', 'warehouse', 'holiday', 'shops_closed',
       'winter_school_holidays', 'school_holidays', 'day', 'month', 'year',
       'day_of_week', 'week_of_year', 'is_weekend', 'is_holiday', 'quarter'],
      dtype='object')
Saving processed train and test files...
Mode: train; datapath: data/processed/train_set_processed.csv, flag: train; features: MS, target: orders, data: custom, batch_size: 16, freq: d, seq_len: 14, label_len: 14, pred_len: 1, embed: timeF
train 7326
	iters: 100, epoch: 1 | loss: 0.1049249
	speed: 0.0137s/iter; left time: 11.1313s
	iters: 200, epoch: 1 | loss: 0.0592419
	speed: 0.0054s/iter; left time: 3.8906s
	iters: 300, epoch: 1 | 

In [3]:
modified_test_set

,date,warehouse,holiday,shops_closed,winter_school_holidays,school_holidays,day,month,year,day_of_week,week_of_year,is_weekend,is_holiday,quarter,orders
0,2024-03-01,1,0,0,0,0,1,3,2024,4,9,0,0,1,7249.0
1,2024-03-02,1,0,0,0,0,2,3,2024,5,9,1,0,1,7103.0
2,2024-03-03,1,0,0,0,0,3,3,2024,6,9,1,0,1,6258.0
3,2024-03-04,1,0,0,0,0,4,3,2024,0,10,0,0,1,6450.0
4,2024-03-05,1,0,0,0,0,5,3,2024,1,10,0,0,1,6575.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,2024-05-11,1,0,0,0,0,11,5,2024,5,19,1,0,2,5000.0
407,2024-05-12,1,0,0,0,0,12,5,2024,6,19,1,0,2,5000.0
408,2024-05-13,1,0,0,0,0,13,5,2024,0,20,0,0,2,5000.0
409,2024-05-14,1,0,0,0,0,14,5,2024,1,20,0,0,2,5000.0


In [4]:
unseen_data, unseen_loader = data_provider(root_path=args['root_dir'],
                                           data_path='data/processed/test_set_processed_modified.csv',
                                           flag='train', features=args['features'],
                                           target=args['target'], data=args['data'],
                                           batch_size=1, freq=args['freq'],
                                           seq_len=args['seq_len'], label_len=args['label_len'],
                                           pred_len=args['pred_len'], embed=args['embed']
                                           )


Mode: train; datapath: data/processed/test_set_processed_modified.csv, flag: train; features: MS, target: orders, data: custom, batch_size: 1, freq: d, seq_len: 14, label_len: 14, pred_len: 1, embed: timeF
train 397


In [5]:

# preds = []
# # prev output
# # outputs = inputs[0][13][13]  # [First sample, last time stamp, last column (orders)]
# device='cuda'
# model.eval()
# with torch.no_grad():
#     for idx, (inputs, targets,  _, _) in enumerate(unseen_loader):
#         inputs = inputs.float().to(device)
#         
#         if idx > 0:
#             total_preds = torch.cat(preds)
#             num_preds = len(total_preds)
#             for prev_pred in range(1, num_preds+1):
#                 inputs[0][-num_preds][13] = preds[prev_pred-1]
#                 num_preds = num_preds - 1
                
            # inputs[0][13][13] = outputs
        
        # if args['use_amp']:
        #     with torch.cuda.amp.autocast():
        #         outputs = model(inputs)
        # else:
        #     outputs = model(inputs)
        #     
        # f_dim = -1 if args['features'] == 'MS' else 0
        # 
        # # Output for t+1
        # outputs = outputs[:, -args['pred_len']:, f_dim:]
        
        # preds.append(outputs)
        # 
        # if idx == 14:
        #     break
        
        

In [6]:
# inputs

In [7]:
# preds = torch.cat(preds)
# preds

In [8]:
preds = []
device = 'cuda'
model.eval()
with torch.no_grad():
    for idx, (inputs, targets, _, _) in enumerate(unseen_loader):
        inputs = inputs.float().to(device)

        # Modify the input with previous predictions after the first batch
        if idx > 0:
            num_preds = min(len(preds), args['seq_len'])
            for i in range(1, num_preds + 1):
                inputs[0, -i, 13] = preds[-i].squeeze()  # Replace the last column with the prediction

        # Make predictions
        if args.get('use_amp', False):
            with torch.cuda.amp.autocast():
                outputs = model(inputs)
        else:
            outputs = model(inputs)

        # Extract the predictions for the next time step(s)
        f_dim = -1 if args['features'] == 'MS' else 0
        outputs = outputs[:, -args['pred_len']:, f_dim:]

        # Store the predictions
        preds.append(outputs)

        # # Optional: Stop after 14 iterations (if that's intended)
        # if idx == 20:
        #     break

# Combine all predictions
total_preds = torch.cat(preds, dim=1).cpu().numpy()


In [9]:
total_preds[0]

array([[5.5039606],
       [5.5148826],
       [5.2664776],
       [5.308286 ],
       [5.571896 ],
       [5.5170436],
       [5.379708 ],
       [5.7942457],
       [5.8216276],
       [6.433873 ],
       [5.6151886],
       [4.961622 ],
       [5.0308523],
       [5.6020713],
       [5.691684 ],
       [5.846496 ],
       [5.738095 ],
       [5.585499 ],
       [5.48652  ],
       [5.562692 ],
       [5.428604 ],
       [5.304237 ],
       [5.412379 ],
       [5.736886 ],
       [5.754243 ],
       [5.6301503],
       [5.5008454],
       [5.406883 ],
       [5.377256 ],
       [5.4954853],
       [5.5645533],
       [5.549763 ],
       [5.5511723],
       [5.6051326],
       [5.5618753],
       [5.464755 ],
       [5.4244065],
       [5.453267 ],
       [5.4950647],
       [5.559495 ],
       [5.5869694],
       [5.5434194],
       [5.490153 ],
       [5.4859476],
       [5.485223 ],
       [5.4753976],
       [5.488644 ],
       [5.52256  ],
       [5.5370646],
       [5.5338974],


In [10]:
final_test = pd.DataFrame(unseen_data.data_x[args['seq_len']:])
print(final_test.shape)
final_test

(397, 14)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.495018,-0.265165,-0.140894,-0.140894,-0.270383,0.057392,-1.327661,0.0,1.080927,-1.638098,1.643990,-0.265165,-1.594711,-0.181595
1,0.495018,-0.265165,-0.140894,-0.140894,-0.270383,0.173020,-1.327661,0.0,1.587496,-1.638098,1.643990,-0.265165,-1.594711,-0.181595
2,0.495018,-0.265165,-0.140894,-0.140894,-0.270383,0.288648,-1.327661,0.0,-1.451917,-1.266953,-0.608276,-0.265165,-1.594711,-0.181595
3,0.495018,-0.265165,-0.140894,-0.140894,-0.270383,0.404277,-1.327661,0.0,-0.945349,-1.266953,-0.608276,-0.265165,-1.594711,-0.181595
4,0.495018,-0.265165,-0.140894,-0.140894,-0.270383,0.519905,-1.327661,0.0,-0.438780,-1.266953,-0.608276,-0.265165,-1.594711,-0.181595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
392,-0.965166,-0.265165,-0.140894,-0.140894,-0.270383,-0.520749,1.463483,0.0,1.080927,1.331068,1.643990,-0.265165,0.627073,-0.181595
393,-0.965166,-0.265165,-0.140894,-0.140894,-0.270383,-0.405121,1.463483,0.0,1.587496,1.331068,1.643990,-0.265165,0.627073,-0.181595
394,-0.965166,-0.265165,-0.140894,-0.140894,-0.270383,-0.289492,1.463483,0.0,-1.451917,1.702214,-0.608276,-0.265165,0.627073,-0.181595
395,-0.965166,-0.265165,-0.140894,-0.140894,-0.270383,-0.173864,1.463483,0.0,-0.945349,1.702214,-0.608276,-0.265165,0.627073,-0.181595


In [11]:
final_test[13] = total_preds[0]
print(final_test.shape)
final_test = final_test.values
final_test

(397, 14)


array([[ 0.4950175 , -0.26516504, -0.14089399, ..., -0.26516504,
        -1.59471109,  5.50396061],
       [ 0.4950175 , -0.26516504, -0.14089399, ..., -0.26516504,
        -1.59471109,  5.51488256],
       [ 0.4950175 , -0.26516504, -0.14089399, ..., -0.26516504,
        -1.59471109,  5.26647758],
       ...,
       [-0.9651657 , -0.26516504, -0.14089399, ..., -0.26516504,
         0.62707283,  5.48750257],
       [-0.9651657 , -0.26516504, -0.14089399, ..., -0.26516504,
         0.62707283,  5.4874649 ],
       [-0.9651657 , -0.26516504, -0.14089399, ..., -0.26516504,
         0.62707283,  5.48742676]])

In [12]:
final_test = unseen_data.inverse_transform(final_test.reshape(-1, final_test.shape[-1])).reshape(final_test.shape)
final_test

array([[4.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 6.99341221e+03],
       [4.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 6.99724156e+03],
       [4.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 6.91014831e+03],
       ...,
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.00000000e+00, 6.98764186e+03],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.00000000e+00, 6.98762866e+03],
       [1.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 2.00000000e+00, 6.98761528e+03]])

In [20]:
test_final_df = pd.DataFrame(final_test)
test_final_df = test_final_df.sort_values(by=[test_final_df.columns[7], test_final_df.columns[8], test_final_df.columns[9]])
test_final_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
2,4.0,0.0,0.0,0.0,0.0,18.0,3.0,2024.0,0.0,12.0,0.0,0.0,1.0,6910.148314
63,0.0,0.0,0.0,0.0,0.0,18.0,3.0,2024.0,0.0,12.0,0.0,0.0,1.0,6993.649616
124,5.0,0.0,0.0,0.0,0.0,18.0,3.0,2024.0,0.0,12.0,0.0,0.0,1.0,6991.183821
185,6.0,0.0,0.0,0.0,0.0,18.0,3.0,2024.0,0.0,12.0,0.0,0.0,1.0,6990.423636
245,3.0,0.0,0.0,0.0,0.0,18.0,3.0,2024.0,0.0,12.0,0.0,0.0,1.0,6989.623829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,4.0,0.0,0.0,0.0,0.0,12.0,5.0,2024.0,6.0,19.0,1.0,0.0,2.0,6998.211227
118,0.0,0.0,0.0,0.0,0.0,12.0,5.0,2024.0,6.0,19.0,1.0,0.0,2.0,6991.199202
179,5.0,0.0,0.0,0.0,0.0,12.0,5.0,2024.0,6.0,19.0,1.0,0.0,2.0,6990.504052
240,6.0,0.0,0.0,0.0,0.0,12.0,5.0,2024.0,6.0,19.0,1.0,0.0,2.0,6989.690368


- **Remove target column when training and testing, then we will have only 13 columns (-2 cuz date and orders)**

In [22]:
solution_df = pd.read_csv('data/solution_example.csv')
solution_df['orders'] = test_final_df[13]
solution_df.to_csv('data/solution_example.csv', index=False)
solution_df

,id,orders
0,Prague_1_2024-03-16,6993.412215
1,Prague_1_2024-03-17,6997.241560
2,Prague_1_2024-03-18,6910.148314
3,Prague_1_2024-03-19,6924.806825
4,Prague_1_2024-03-20,7017.231062
...,...,...
392,Budapest_1_2024-05-11,6987.668447
393,Budapest_1_2024-05-12,6987.655239
394,Budapest_1_2024-05-13,6987.641865
395,Budapest_1_2024-05-14,6987.628657
